<a href="https://colab.research.google.com/github/marianefn/Project_Lattices/blob/main/Project_Lattices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Θέμα 8

## Βρες αν το διάνυσμα (43, 69, 95, 110) ανήκει στο πλέγμα

In [ ]:
import numpy as np

# Ορισμός των πινάκων A και του διανύσματος v
A = np.array([[1, 2, 3, 4],
              [-1, 1, 2, 3],
              [4, 3, 2, 1],
              [0, 0, 1, 1]])

B = np.array([[5, 8, 11, 13],
              [4, 6, 8, 9],
              [8, 9, 10, 10],
              [4, 6, 9, 10]])

v = np.array([43, 69, 95, 110])

# Λύση για το σύστημα εξισώσεων A * x = v (γραμμικός συνδυασμός των γραμμών του A)
coefficients_A, resids, rank, s = np.linalg.lstsq(A.T, v, rcond=None)

# Έλεγχος αν το διάνυσμα ανήκει στο πλέγμα του A (αν τα υπόλοιπα είναι πολύ μικρά)
print("Συντελεστές για τις γραμμές του πίνακα A:", coefficients_A)
print("Υπόλοιπα για A:", resids)

# Λύση για το σύστημα εξισώσεων B * x = v (γραμμικός συνδυασμός των γραμμών του B)
coefficients_B, resids_b, rank_b, s_b = np.linalg.lstsq(B.T, v, rcond=None)

# Έλεγχος αν το διάνυσμα ανήκει στο πλέγμα του B (αν τα υπόλοιπα είναι πολύ μικρά)
print("\nΣυντελεστές για τις γραμμές του πίνακα B:", coefficients_B)
print("Υπόλοιπα για B:", resids_b)


Συντελεστές για τις γραμμές του πίνακα A: [14. 11. 10. 11.]
Υπόλοιπα για A: []

Συντελεστές για τις γραμμές του πίνακα B: [ 3.00000000e+00  9.00000000e+00 -1.00000000e+00 -4.17443857e-14]
Υπόλοιπα για B: []


## Βρες ορίζουσα πίνακα Α

In [ ]:
import numpy as np

# Ορισμός του πίνακα A (ή B)
A = np.array([[1, 2, 3, 4],
              [-1, 1, 2, 3],
              [4, 3, 2, 1],
              [0, 0, 1, 1]])

# Υπολογισμός του όγκου του πλέγματος (ορίζουσα του πίνακα A)
volume_A = np.abs(np.linalg.det(A))

print("Ο όγκος του πλέγματος που παράγεται από τις γραμμές του πίνακα A είναι:", volume_A)

Ο όγκος του πλέγματος που παράγεται από τις γραμμές του πίνακα A είναι: 5.000000000000001


## Σημεία πλέγματος μέσα σε κύκλο και τετράγωνο

In [ ]:
import numpy as np

# Ορισμός του πίνακα A
A = np.array([[1, 2, 3, 4],
              [-1, 1, 2, 3],
              [4, 3, 2, 1],
              [0, 0, 1, 1]])

B = np.array([[5, 8, 11, 13],
              [4, 6, 8, 9],
              [8, 9, 10, 10],
              [4, 6, 9, 10]])


# Συνάρτηση για να ελέγξουμε αν ένα διάνυσμα ανήκει στο πλέγμα
def is_in_lattice(vector, matrix):
    try:
        # Λύνουμε το σύστημα A * c = vector
        coefficients, resids, rank, s = np.linalg.lstsq(matrix.T, vector, rcond=None)
        # Ελέγχουμε αν τα υπόλοιπα (resids) είναι κοντά στο μηδέν
        if not np.allclose(np.dot(matrix.T, coefficients), vector):
            return False

        # Ελέγχουμε αν όλοι οι συντελεστές είναι ακέραιοι
        return np.allclose(coefficients, np.round(coefficients))
    except:
        return False

# Συνάρτηση για να υπολογίσουμε τα σημεία εντός του κύκλου
def points_in_circle(radius, matrix):
    count = 0
    for x in range(-radius, radius + 1):
        for y in range(-radius, radius + 1):
            if x**2 + y**2 < radius**2:  # Εντός του κύκλου
                vector = np.array([x, y, 0, 0])  # Μορφή (x, y, 0, 0)
                if is_in_lattice(vector, matrix):
                    count += 1
    return count

# Συνάρτηση για να υπολογίσουμε τα σημεία εντός του τετραγώνου
def points_in_square(side_length, matrix):
    count = 0
    half_side = side_length // 2
    for x in range(-half_side, half_side + 1):
        for y in range(-half_side, half_side + 1):
          if abs(x + y) + abs(x - y) < side_length:
            vector = np.array([x, y, 0, 0])  # Μορφή (x, y, 0, 0)
            if is_in_lattice(vector, matrix):
                count += 1
    return count

# Ρυθμίσεις
radius = 10  # Ακτίνα του κύκλου
side_length = 10  # Πλευρά του τετραγώνου

# Υπολογισμός
points_circle = points_in_circle(radius, B)
points_square = points_in_square(side_length, B)

print(f"Σημεία του πλέγματος μέσα στον κύκλο με ακτίνα 10: {points_circle}")
print(f"Σημεία του πλέγματος μέσα στο τετράγωνο με πλευρά 10: {points_square}")


Σημεία του πλέγματος μέσα στον κύκλο με ακτίνα 10: 61
Σημεία του πλέγματος μέσα στο τετράγωνο με πλευρά 10: 17


# Θέμα 3

In [23]:
import numpy as np

def are_linearly_independent(vec1, vec2):
    """
    Ελέγχει αν δύο διανύσματα είναι γραμμικώς ανεξάρτητα.

    :param vec1: Το πρώτο διάνυσμα (λίστα ή numpy array)
    :param vec2: Το δεύτερο διάνυσμα (λίστα ή numpy array)
    :return: True αν είναι γραμμικώς ανεξάρτητα, False αν όχι

    ~~ function created with chatgpt
    """
    # Μετατροπή σε numpy arrays για ευκολία
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)

    # Έλεγχος αν τα διανύσματα έχουν ίδιο μέγεθος
    if vec1.shape != vec2.shape:
        raise ValueError("Τα διανύσματα πρέπει να έχουν το ίδιο μέγεθος.")

    # Δημιουργία πίνακα από τα δύο διανύσματα
    matrix = np.column_stack((vec1, vec2))

    # Υπολογισμός ορίζουσας
    determinant = np.linalg.det(matrix)

    # Τα διανύσματα είναι γραμμικώς ανεξάρτητα αν η ορίζουσα δεν είναι 0
    return not np.isclose(determinant, 0)

# Παράδειγμα χρήσης
vec1 = [1, 1]
vec2 = [3, 4]

print(are_linearly_independent(vec1, vec2))


True


In [24]:
import numpy as np

def calc_norm_squared(vec):
  """Calculates the Euclidean norm of a vector squared up.
  :param vec: A list or numpy array representing the vector.
  :return: The Euclidean norm of the vector squared up.
  """
  vector = np.array(vec)
  return np.sum(vector**2)

In [25]:
# prompt: create function to calculate the norm of a vector (gemini)

def calc_norm(vec):
  """Calculates the Euclidean norm of a vector.
  :param vec: A list or numpy array representing the vector.
  :return: The Euclidean norm of the vector.
  """
  vector = np.array(vec)
  return np.linalg.norm(vector)

In [26]:
# prompt: create function to swap vectors represented by numpy use an example to swap two vectors

import numpy as np

def swap_vectors(vec1, vec2):
    """Swaps two vectors represented by NumPy arrays.
    :param vec1: The first vector (NumPy array).
    :param vec2: The second vector (NumPy array).
    :return: A tuple containing the swapped vectors (vec2, vec1).
    """
    return vec2, vec1

In [27]:
import numpy as np

def angle_between_vectors(vec1, vec2):
    """ Calculates the angle between two vectors in Z^2.
    :param v1: Το πρώτο διάνυσμα (λίστα ή numpy array)
    :param v2: Το δεύτερο διάνυσμα (λίστα ή numpy array)
    :return: Η γωνία σε μοίρες
    """
    # Transform into numpy arrays
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)

    # Calculation of dot product
    dot_product = np.dot(vec1, vec2)

    # Calculation of norms of the two vectors
    norm_v1 = np.linalg.norm(vec1)
    norm_v2 = np.linalg.norm(vec2)

    # Calculation of cos(θ)
    cos_theta = dot_product / (norm_v1 * norm_v2)

    # Check that cos(θ) in in [-1, 1] to avoid numeric errors
    cos_theta = np.clip(cos_theta, -1, 1)

    # Calculation in radians
    theta_radians = np.arccos(cos_theta)

    # Transformation in degrees
    theta_degrees = np.degrees(theta_radians)

    return theta_degrees

# Παράδειγμα χρήσης
v1 = [1, 2]
v2 = [3, 4]

angle = angle_between_vectors(v1, v2)
print(f"Η γωνία μεταξύ των διανυσμάτων είναι: {angle:.2f} μοίρες")


Η γωνία μεταξύ των διανυσμάτων είναι: 10.30 μοίρες


In [21]:
# prompt: create function that generates a vector in Z^2 without limitations

import numpy as np

def generate_vector_z2():
    """Generates a random vector in Z^2."""
    x = np.random.randint(-100, 101)
    y = np.random.randint(-100, 101)
    return np.array([x, y])

# Example usage
vector = generate_vector_z2()
vector

array([-33, -14])

In [37]:
def create_vector_pairs():
  """ Finds two random linear independent vectors in Z^2
  :return: vec1, vec2"""

  found = False
  vec1 = generate_vector_z2()
  while not found:
    vec2 = generate_vector_z2()
    if are_linearly_independent(vec1,vec2):
      found = True

  return vec1, vec2

vec1, vec2 = create_vector_pairs()
print(vec1,vec2)

[ 74 -50] [75 56]


## Αλγόριθμος των Gauss-Lagrange

In [44]:
import math


def gauss_lagrange(vec1, vec2):
  """Outputs a Lagrange-Gauss reduced basis for the lattice L.
    :param vec1: The first vector (NumPy array).
    :param vec2: The second vector (NumPy array).
    :return: Basis (b1, b2) for L such that ||b_i|| = λ_i
    """

  b1 = calc_norm_squared(vec1)
  m = np.dot(vec1, vec2)/b1
  vec2 = vec2 - math.ceil(m-0.5)*vec1
  b2 = calc_norm_squared(vec2)

  while calc_norm(vec2) < calc_norm(vec1):
      vec1, vec2 = swap_vectors(vec1, vec2)
      b1 = b2
      m = np.dot(vec1, vec2)/b1
      vec2 = vec2 - math.ceil(m-0.5)*vec1
      b2 = calc_norm_squared(vec2)

  return vec1, vec2


# Requesting the vectors as input
while True:
    try:
        vec1_str = input("Enter the first vector (e.g., 1,2,3): ")
        vec1 = np.array([int(x) for x in vec1_str.split(',')])
        vec2_str = input("Enter the second vector (e.g., 4,5,6): ")
        vec2 = np.array([int(x) for x in vec2_str.split(',')])

        if not are_linearly_independent(vec1,vec2):
            print("Vectors must have the same dimension. Please enter again.")
            continue

        break
    except ValueError:
        print("Invalid input. Please enter numbers separated by commas.")

# Checking linear independence
if are_linearly_independent(vec1, vec2):
    print("The vectors are linearly independent.")
else:
    print("The vectors are linearly dependent.")

GL_vec1, GL_vec2 = gauss_lagrange(vec1, vec2)
print("Gauss-Lagrange b1 =",GL_vec1)
print("Gauss-Lagrange b2 =",GL_vec2)

l1 = calc_norm(GL_vec1)
print("l1 =",l1)



Enter the first vector (e.g., 1,2,3): 1,3
Enter the second vector (e.g., 4,5,6): 3,4
The vectors are linearly independent.
Gauss-Lagrange b1 = [2 1]
Gauss-Lagrange b2 = [-1  2]
l1 = 2.23606797749979


In [ ]:
angle_list = []

for i in range(50):
  # find two linearly independent vectors in Z^2
  vec1, vec2 = create_vector_pairs()
  # calculate the Gauss-Lagrange vectors
  GL_vec1, GL_vec2 = gauss_lagrange(vec1, vec2)

  # Calulate the angle between the two vectors
  angle = angle_between_vectors(GL_vec1, GL_vec2)
  angle_list.append(angle)


angle_array = np.array(angle_list)
print(angle_array)
print(angle_array.mean())



[ 94.81148364  93.44085692 114.71453102  91.39718103  90.
  75.48573371 100.01469127  96.00900596  81.0342     101.63170046
 114.35425641  92.51844016  91.73570459  80.71444145  70.29977509
  94.72013693 105.09487611  87.51044708  89.56921278 111.66407567
  85.68397248  76.97080519  97.97427377  77.86027484  90.
  92.1210964  118.45591837  90.27381287 100.05153527  97.28985398
  87.51044708  77.33998136  90.16884758  90.11725476  99.67091475
  79.39351213  78.59540731  94.82468843  71.14576364  94.9778499
  80.20270283 111.61477894  87.41643746  87.48312434  71.38983527
  88.94478647  99.33232633  84.39290016  72.82206566  96.59991698]
90.94691669633356


In [ ]:
# prompt: find average of numpy array

import numpy as np

angle_array = np.array(angle_list)
print(angle_array.mean())

# Θέμα 4

## Αλγόριθμος Gram-Schmidt

In [11]:
import numpy as np

def gram_schmidt(basis):
    """
    Gram-Schmidt Algorithm to create an orthogonal basis.
    :param basis: a list of vecotrs (NumPy arrays) which create the basis B.
    :return orthogonal_basis: list of orthogonal vectors
    :return mu: Gram-Schmidt matrix (numpy array) with μ_ij cofacotrs

    ~ used chatgpt as a reference
    """
    # Initialize the orthogonal basis and the Gram-Schmidt matrix
    n = len(basis)
    orthogonal_basis = []
    mu = np.zeros((n, n))

    for i in range(n):
      t1 = basis[i]
      #t2 = t1.copy()
      t2 = t1.astype(np.float64)

      for j in range(i):
        #mu[i, j] = np.dot(t1, orthogonal_basis[j]) / np.dot(orthogonal_basis[j], orthogonal_basis[j])
        # Perform integer division and round to nearest integer
        mu[i, j] = np.round(np.dot(t1, orthogonal_basis[j]) / np.dot(orthogonal_basis[j], orthogonal_basis[j])).astype(int)
        t2 -= mu[i, j] * orthogonal_basis[j]

      orthogonal_basis.append(t2)

    return orthogonal_basis, mu

# Παράδειγμα χρήσης
basis = [
    np.array([1.0, 1.0]),
    np.array([1.0, 0.0])
]

orthogonal_basis, mu = gram_schmidt(basis)

# Εκτύπωση αποτελεσμάτων
print("Ορθογώνια βάση:")
for vec in orthogonal_basis:
    print(vec)

print("\nGram-Schmidt πίνακας (μ):")
print(mu)


Ορθογώνια βάση:
[1. 1.]
[1. 0.]

Gram-Schmidt πίνακας (μ):
[[0. 0.]
 [0. 0.]]


## Αλγόριθμος LLL

In [50]:
import math
import numpy as np

def lll_algorithm(basis, delta):
  """
    LLL algorithm with Euclidean norm
    :param orthogonal_basis: a list of vecotrs (NumPy arrays) which the basis B.
    :param delta: typically choose δ = 3/4
    :return orthogonal_basis: list of orthogonal vectors
    :return mu: LLL reduced basis b1, . . . , bn (numpy array)
  """
  n = len(basis)
  orthogonal_basis, mu = gram_schmidt(basis)
  # Compute Bi = <{b_i}*,{b_i}*> = ||{b_i}*||^2
  B = np.array([np.dot(orthogonal_basis[i], orthogonal_basis[i]) for i in range(n)])

  k=1
  while k < n:
    # Perform size reduction
    for j in range(k - 1, -1, -1):
      q = math.floor(mu[k,j]+0.5)
      basis[k] -= q*basis[j]
      mu[k, :k + 1] -= q * mu[j, :k + 1]

    orthogonal_basis, mu = gram_schmidt(basis)
    B = np.array([np.dot(orthogonal_basis[i], orthogonal_basis[i]) for i in range(n)])

    # Check Lov´asz condition
    if k>0 and B[k] < (delta-mu[k,k-1]**2)*B[k-1]:
        # Swap basis[k] and basis[k-1]
        basis[k], basis[k-1] = basis[k-1], basis[k].copy()
        orthogonal_basis, mu = gram_schmidt(basis)
        B = np.array([np.dot(orthogonal_basis[i], orthogonal_basis[i]) for i in range(n)])
        k = max(1, k - 1) # Changed from max(2, k - 1) to max(1, k - 1)
    else:
      k+=1

  return basis



# Examples
delta = 0.75
basis = np.array([[1, 1, 1], [1, 0, 2], [0, 1, 1]])
lll_reduced_basis = lll_algorithm(basis,delta)
print("LLL Reduced Basis:")
print(lll_reduced_basis)

basis = np.array([[10,1], [1, 9]])
lll_reduced_basis = lll_algorithm(basis,delta)
print("LLL Reduced Basis:")
print(lll_reduced_basis)

basis = np.array([[1, -1, 3], [1, 0, 5], [1, 2, 6]])
lll_reduced_basis = lll_algorithm(basis,delta)
print("LLL Reduced Basis:")
print(lll_reduced_basis)

basis = np.array([[-36,32], [-32, -27]])
lll_reduced_basis = lll_algorithm(basis,delta)
print("LLL Reduced Basis:")
print(lll_reduced_basis)

basis = np.array([[-1, -1, -2, -4, -1], [-2, -2, 4, -4, 2], [4, 2, -1, 1, 4], [1, -1, 1, -1, -1],[-1, 1, -1, 1, -30]])
lll_reduced_basis = lll_algorithm(basis,delta)
print("LLL Reduced Basis:")
print(lll_reduced_basis)

basis = np.array([[42, -36, -5], [-13, 9, 30]])
lll_reduced_basis = lll_algorithm(basis,delta)
print("LLL Reduced Basis:")
print(lll_reduced_basis)


LLL Reduced Basis:
[[-1  0  0]
 [ 0 -1  1]
 [ 0  1  1]]
LLL Reduced Basis:
[[10  1]
 [ 1  9]]
LLL Reduced Basis:
[[ 1 -1  0]
 [-1  0  1]
 [ 1  1  1]]
LLL Reduced Basis:
[[-36  32]
 [-32 -27]]
LLL Reduced Basis:
[[ 1 -1  1 -1 -1]
 [-2  0 -3 -3  0]
 [-3 -1  3 -3  3]
 [ 5  1  0  0  3]
 [ 0  6  3 -3 -7]]
LLL Reduced Basis:
[[-13   9  30]
 [ 29 -27  25]]


## πρώτο διάνυσμα που δίνει ο LLL έχει μεγαλύτερο μήκος από το δεύτερο διάνυσμα

In [36]:
def check_lll_norm_condition(max_iterations=100):
  """
  Checks random lattices (up to max_iterations) until it finds one where the
  norm of the first vector of the LLL-reduced lattice is greater than the second.
  :parm max_iterations: The maximum number of iterations to perform.
  :returns a tuple containing the original base, the LLL-reduced basis and the norms if the condition is met,
    None otherwise.

  ~ used Gemini as reference for naming
  """

  for _ in range(max_iterations):
    # Generate two random linearly independent vectors in Z^2
    vec1, vec2 = create_vector_pairs()
    basis = np.array([vec1, vec2])

    # Apply the LLL algorithm
    lll_basis = lll_algorithm(basis, delta=0.75)

    # Check the condition
    norm_first = calc_norm(lll_basis[0])
    norm_second = calc_norm(lll_basis[1])

    if norm_first > norm_second:
        return  basis, lll_basis, (norm_first, norm_second)

  return None


result = check_lll_norm_condition()

if result:
    basis, lll_basis, norms = result
    print("Original basis found satisfying the condition:")
    print(basis)
    print("LLL-reduced basis found satisfying the condition:")
    print(lll_basis)
    print("Norms:", norms)
else:
    print("No LLL-reduced basis found satisfying the condition within the given iterations.")

Original basis found satisfying the condition:
[[-42 -93]
 [ 93   1]]
LLL-reduced basis found satisfying the condition:
[[-42 -93]
 [ 93   1]]
Norms: (102.04410811016969, 93.00537618869137)


## Τυχαία εκτέλεση LLL και Gauss-Lagrange σε πλέγματα βαθμίδας 2

In [42]:
def random_lll_executions(iterations):
  """
  Checks <iterations> random lattices
  :parm iterations: The number of iterations to perform.
  :returns a tuple containing the original base, the LLL-reduced basis
  """

  for _ in range(iterations):
    # Generate two random linearly independent vectors in Z^2
    vec1, vec2 = create_vector_pairs()
    basis = np.array([vec1, vec2])

    # Apply the LLL algorithm
    lll_basis = lll_algorithm(basis, delta=0.75)
    print("Original basis:")
    print(basis)
    print("LLL-reduced basis:")
    print(lll_basis)

  return None


run = random_lll_executions(2)

Original basis:
[[  8   6]
 [ 30 -32]]
LLL-reduced basis:
[[  8   6]
 [ 30 -32]]
Original basis:
[[ 22  47]
 [ 42 -24]]
LLL-reduced basis:
[[ 22  47]
 [ 42 -24]]


In [47]:
def random_gauss_lagrange_executions(iterations):
  """
  Checks <iterations> random lattices
  :parm iterations: The number of iterations to perform.
  :returns a tuple containing the original base, the Gauss-Lagrange basis
  """

  for _ in range(iterations):
    # Generate two random linearly independent vectors in Z^2
    vec1, vec2 = create_vector_pairs()
    basis = np.array([vec1, vec2])

    # Apply the Gauss-Lagrance algorithm
    GL_vec1, GL_vec2 = gauss_lagrange(vec1, vec2)
    gauss_basis = np.array([GL_vec1, GL_vec2])
    print("Original basis:")
    print(basis)
    print("Gauss-Lagrange-reduced basis:")
    print(gauss_basis)

  return None


run = random_gauss_lagrange_executions(10)

Original basis:
[[ 64  41]
 [ 27 -78]]
Gauss-Lagrange-reduced basis:
[[ 64  41]
 [ 27 -78]]
Original basis:
[[  0 -55]
 [ 55  16]]
Gauss-Lagrange-reduced basis:
[[  0 -55]
 [ 55  16]]
Original basis:
[[  29   62]
 [-100   79]]
Gauss-Lagrange-reduced basis:
[[  29   62]
 [-100   79]]
Original basis:
[[ 98 -21]
 [ 26  40]]
Gauss-Lagrange-reduced basis:
[[ 26  40]
 [ 72 -61]]
Original basis:
[[-40 -51]
 [-94  10]]
Gauss-Lagrange-reduced basis:
[[-40 -51]
 [-54  61]]
Original basis:
[[-31 -27]
 [-10 -66]]
Gauss-Lagrange-reduced basis:
[[-31 -27]
 [ 21 -39]]
Original basis:
[[-63 -82]
 [-70  14]]
Gauss-Lagrange-reduced basis:
[[-70  14]
 [  7 -96]]
Original basis:
[[ 10 -75]
 [-20  26]]
Gauss-Lagrange-reduced basis:
[[-20  26]
 [-30 -23]]
Original basis:
[[ -9 -13]
 [ -4 -75]]
Gauss-Lagrange-reduced basis:
[[ -9 -13]
 [ 32 -23]]
Original basis:
[[84 96]
 [90 46]]
Gauss-Lagrange-reduced basis:
[[  6 -50]
 [ 96  -4]]
